In [1]:
import pandas as pd
import sys
sys.path.insert(0,r"C:\Users\LENOVO\Downloads")
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)

In [3]:
base_sur = pd.read_csv('base_centro.txt', sep = ",", engine = "python")

In [4]:
base2_sur = base_sur[['SUSCNITC','SERVDESC','SESUCUSA','SESUCICL','SESULOCA',
                                  'TIPIFICACION_CLIENTE','CALIFICACION_VECTOR_SERVICIO',
                                  'CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR',
                                  'ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN','PROM_SUSC']]
base3_sur = base2_sur.dropna()

In [5]:
tmp = base3_sur[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
tmp.columns

group_nit = tmp.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()
group_nit = group_nit.rename(columns={"CALIFICACION_VECTOR_SERVICIO":"PROM_CALIFICACION"})

In [6]:
base4_sur = pd.merge(base3_sur, group_nit, how='left', on=['SUSCNITC'])

In [7]:
base4_sur['y'] = pd.cut(x=base4_sur['PROM_CALIFICACION'],
                            bins=[-1,50,76,100],
                            labels=['No Pago','Pago Inoportuno','Pago'])

In [8]:
base5_sur = base4_sur[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE',
                       'CALIFICACION_VECTOR_SERVICIO','CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO',
                       'DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN','PROM_SUSC','y']]

In [9]:
names = base5_sur.columns

for x in names:
    globals()['FREQ_%s' % x]= base5_sur[x].value_counts().rename_axis([x]).reset_index(name='F_{x}'.format(x=x))

In [10]:
freq_sur = base5_sur.merge(FREQ_SERVDESC,on="SERVDESC").merge(FREQ_SESUCUSA,on='SESUCUSA').merge(FREQ_SESUCICL,on='SESUCICL').merge(FREQ_SESULOCA,on='SESULOCA').merge(FREQ_TIPIFICACION_CLIENTE,on='TIPIFICACION_CLIENTE').merge(FREQ_CANAL_ENVIO,on='CANAL_ENVIO').merge(FREQ_SEGMENTACION,on='SEGMENTACION').merge(FREQ_ESTRATO_AGRUPADO,on='ESTRATO_AGRUPADO').merge(FREQ_DEPARTAMENTO_AGR,on='DEPARTAMENTO_AGR').merge(FREQ_CANAL_PAGO,on='CANAL_PAGO')

In [11]:
freq_sur2 = freq_sur[['F_SERVDESC', 'F_SESUCUSA',
       'F_SESUCICL', 'F_SESULOCA', 'F_TIPIFICACION_CLIENTE', 'F_CANAL_ENVIO',
       'F_SEGMENTACION', 'F_ESTRATO_AGRUPADO', 'F_DEPARTAMENTO_AGR',
       'F_CANAL_PAGO']]

In [12]:
freq_sur2['AVG_FREQ'] = freq_sur2.mean(axis=1)

C:\Users\Paola Fajardo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
n, m = freq_sur2.shape
n_reg_del = round((0.05 * n) /100)
outliers = freq_sur2.nsmallest(n_reg_del, 'AVG_FREQ')
outliers_idx = outliers.index
outliers_idx = outliers_idx.sort_values(ascending=False)

In [14]:
for x in outliers_idx:
    base5_sur = base5_sur.drop([x])

In [15]:
#base5_sur = base5_sur.drop([332839])

In [16]:
base5_sur.shape

(441018, 16)

In [17]:
dataTypeSeries = base5_sur.dtypes
dataTypeSeries

cat = base5_sur.select_dtypes(include=['object','category'])
cuanti = base5_sur.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))
base5_sur = cuanti.join(cat)

In [18]:
X = base5_sur[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE','CANAL_ENVIO',
               'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO',
               'SESUSAPE','SESUSAAN','PROM_SUSC']]

y = base5_sur['y']

In [19]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.870619  0.861996  0.866286  13630.0  13495.0  0.956609
1             0.868285  0.901558  0.884609  46972.0  48772.0  0.916214
2             0.787832  0.740309  0.763331  27602.0  25937.0  0.878609
avg / total   0.843469  0.844984  0.843826  88204.0  88204.0  0.930182
DecisionTreeClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.961704  0.753558  0.845002  13630.0  10680.0  0.950087
1             0.852133  0.911437  0.880788  46972.0  50241.0  0.919149
2             0.766631  0.757771  0.762175  27602.0  27283.0  0.890630
avg / total   0.842308  0.838953  0.838140  88204.0  88204.0  0.943633
RandomForestClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.914030  0.726999  0.809857  13630.0  10841.0  0.9

In [20]:
classifiers = [
    KNeighborsClassifier(2),
    KNeighborsClassifier(3),
    KNeighborsClassifier(4),
    KNeighborsClassifier(5),
    KNeighborsClassifier(6),
    KNeighborsClassifier(7),
    KNeighborsClassifier(8),
    KNeighborsClassifier(9),
    KNeighborsClassifier(10)
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.806162  0.908070  0.854087  13630.0  15353.0  0.946219
1             0.828245  0.936175  0.878909  46972.0  53093.0  0.900838
2             0.850339  0.608688  0.709502  27602.0  19758.0  0.859199
avg / total   0.831746  0.829350  0.822060  88204.0  88204.0  0.914743
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.870619  0.861996  0.866286  13630.0  13495.0  0.956609
1             0.868285  0.901558  0.884609  46972.0  48772.0  0.916214
2             0.787832  0.740309  0.763331  27602.0  25937.0  0.878609
avg / total   0.843469  0.844984  0.843826  88204.0  88204.0  0.930182
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.875465  0.880411  0.877931  13630.0  13707.0  0.96304

In [24]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
X_ = StandardScaler().fit_transform(X_fin)

for train_index, test_index in kf.split(X_, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(7).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                      y_score = model.predict_proba(X_test)))
 
## Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score  support     pred       AUC
0             0.813351  0.771971  0.792121  13700.0  13003.0  0.933456
1             0.825542  0.902162  0.862153  46761.0  51101.0  0.898616
2             0.741992  0.644559  0.689852  27743.0  24100.0  0.838670
avg / total   0.797369  0.800916  0.797081  88204.0  88204.0  0.912837,              precision    recall  f1-score  support     pred       AUC
0             0.797875  0.778191  0.787910  13701.0  13363.0  0.938443
1             0.816008  0.910907  0.860850  46760.0  52198.0  0.898583
2             0.742746  0.606207  0.667566  27743.0  22643.0  0.837586
avg / total   0.790148  0.794454  0.788726  88204.0  88204.0  0.912602,              precision    recall  f1-score  support     pred       AUC
0             0.693695  0.780584  0.734579  13700.0  15416.0  0.916222
1             0.841974  0.907571  0.873542  46760.0  50403.0  0.913109
2             0.739245  0.596453  0.660217  27744.0  22385.0  0.808158
avg

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[ 9904,  1975,  1821],
       [  458, 42531,  3771],
       [ 2291,  9032, 16420]], dtype=int64)